In [11]:
import numpy as np
import pandas as pd
import chardet as cd

In [6]:
# path to file
path = 'BRAINT/sourcedata/sub-002/ses-02/sub-002_ses-02_task-03ArchiLocalizer_23_03_21_14_07.txt'

In [12]:
# to determine the encoding, which is not standard here

with open(path, 'rb') as f:
    result = cd.detect(f.read())  # or readline if the file is large
    
print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [18]:
df_orig = pd.read_csv(path, encoding='ISO-8859-1', delimiter='\t')

print(df_orig)

     CONDITIONS  DURATIONS STIM_TEXT STIM_BITMAP STIM_WAV  RESPONSE_BUTTON  \
0             0         52       NaN        Noir      NaN                0   
1             8         27       NaN        Noir    calc1                0   
2             0          6       NaN        Noir      NaN                0   
3             8         27       NaN        Noir    calc2                0   
4             0         18       NaN        Noir      NaN                0   
..          ...        ...       ...         ...      ...              ...   
484           5         18       NaN        Noir      NaN                4   
485          10         27       NaN        Noir      ph9                0   
486           0         21       NaN        Noir      NaN                0   
487          10         27       NaN        Noir     ph10                0   
488           0         83       NaN        Noir      NaN                0   

     ONSETS_TRIGGERS  ONSETS_MS  REPONSE_1  JUSTE  INSTANT_REPO

In [57]:
# DataFrame for events: condition, onset, duration; time in seconds for nilearn
df_ev = pd.DataFrame()

# previous condition to check blocks
prev_cond = ''
# initialize duration and onset (in seconds)
duration = -1.0
onset = -1.0
onset2 = -1.0

# iterate over rows348156
for l in df_orig[['CONDITIONS', 'ONSETS_MS', 'ONSETS_TRIGGERS', 'DURATIONS']].iterrows():
    # get values
    _, (cur_cond, cur_ons2, cur_ons, cur_dur) = l
    # check if equality between current and past values for condition
    if cur_cond==prev_cond:
        duration += cur_dur
    else:
        # add row to DataFrame (conversion of onset and duration from trigger to seconds, 1 trigger = 80 ms; conversion of onset 2 from ms to s)
#        print(cur_cond, onset, duration)
        if not prev_cond=='':
            df_tmp = pd.DataFrame({'condition': prev_cond, 'onset2': onset2 / 1000.0, 'onset': onset * 0.08, 'duration': duration * 0.08}, [0])
            df_ev = pd.concat((df_ev, df_tmp), ignore_index=True)
        # move to next block
        prev_cond = cur_cond
        onset2 = cur_ons2
        onset = cur_ons
        duration = cur_dur
    

In [59]:
# check if onsets are similar in ms and trigger
df_ev

,condition,onset2,onset,duration
0,0,0.000,0.00,4.16
1,8,4.160,4.16,2.16
2,0,6.320,6.32,0.48
3,8,6.800,6.80,2.16
4,0,8.960,8.96,3.60
...,...,...,...,...
153,0,338.076,338.08,1.36
154,5,339.436,339.44,2.72
155,10,342.157,342.16,2.16
156,0,344.316,344.32,1.68


In [54]:
# check all conditions
print(np.unique(df_ev['condition']))

[ 0  1  2  3  4  5  6  7  8  9 10]


In [60]:
print(df_ev[['condition', 'onset', 'duration']])

     condition   onset  duration
0            0    0.00      4.16
1            8    4.16      2.16
2            0    6.32      0.48
3            8    6.80      2.16
4            0    8.96      3.60
..         ...     ...       ...
153          0  338.08      1.36
154          5  339.44      2.72
155         10  342.16      2.16
156          0  344.32      1.68
157         10  346.00      2.16

[158 rows x 3 columns]
